In [130]:
from synbiolib import codon
import pandas as pd
import itertools
import os
from opentrons import robot

In [2]:
original_df = pd.read_excel("./../../data/orders/8491455_SapI-Synthesis.xls")

,Well,Primer,Seq,Block
0,A1,KG_1,CGCAATTCCACCACAGAAGACTAGGAGGCCCAGCGCTATGTCTTCG...,NOT_BLOCK
1,A2,KG_2,CTGGCGAGACAAGGCGAAGACATAGCGCTGGGCCTCCTAGTCTTCT...,NOT_BLOCK
2,A3,KG_3,CGCAATTCCACCACAGAAGACTAGGAGGCCCAG,NOT_BLOCK
3,A4,KG_4,CTGGCGAGACAAGGCGAAGACATAGCGCTGGGC,NOT_BLOCK
4,A5,KG_5,CAGCGCTATGTCTTCGCCTTGTCTCGCCAG,NOT_BLOCK
5,A6,KG_6,GGCCTCCTAGTCTTCTGTGGTGGAATTGCG,NOT_BLOCK
6,A7,KG_7,CGCAATTCCACCACAGAAGACTAGGAGGCC,NOT_BLOCK
7,A8,KG_8,CTGGCGAGACAAGGCGAAGACATAGCGCTG,NOT_BLOCK
8,A9,KG_9,CGCTATGTCTTCGCCTTGTCTCGCCAG,NOT_BLOCK
9,A10,KG_10,CTCCTAGTCTTCTGTGGTGGAATTGCG,NOT_BLOCK


In [89]:
forward_attachment = "AGATGGCTCTTCT"
reverse_attachment = "TGAAGAGCCACGG"

def block_to_oligo(block_seq):
    for_primer = forward_attachment + block_seq + reverse_attachment
    rev_primer = codon.reverse_complement(for_primer)
    return (for_primer, rev_primer)

def oligo_to_block(oligo_seq):
    if forward_attachment in oligo_seq:
        block = oligo_seq[len(forward_attachment):len(forward_attachment)+5]
    elif codon.reverse_complement(forward_attachment) in oligo_seq:
        block = codon.reverse_complement(oligo_seq[len(reverse_attachment):len(reverse_attachment)+5])
    else:
        block = "NOT_BLOCK"
    return block      
def synseq_to_blocks(oligo): # synseq_to_blocks("GCCATGGCTAGCAAAGCAG")
    blocks = []
    for i,c in enumerate(oligo):
        start = i * 2
        block = oligo[start:start+5]
        if len(block) < 5:
            break
        blocks.append(block)
    return blocks
def find_wells(term,query): # 'Block','GCAAC'
    return list(df.loc[df[term] == query]['Well'])

def blocks_in_dataframe(df):
    return list(set(df[df['Block'] != 'NOT_BLOCK']['Block']))

def change_block(seq,block_name):
    return df.loc[df['Seq'] == seq, 'Block'] = block_name

SyntaxError: invalid syntax (<ipython-input-89-f53c31163595>, line 33)

In [96]:
df = original_df.iloc[3:51][["Order Number:",8491455,"Unnamed: 3"]].reset_index(drop=True)
df.columns = ["Well", "Primer", "Seq"]
df["Block"] = df["Seq"].apply(oligo_to_block)

for seq in ["CGCAATTCCACCACAGAAGACTAGGAGGCCCAGCGCTATGTCTTCGCCTTGTCTCGCCAG", "CTGGCGAGACAAGGCGAAGACATAGCGCTGGGCCTCCTAGTCTTCTGTGGTGGAATTGCG"]:
    change_block(seq,"POS_CTRL")

for seq in ["CGCAATTCCACCACAGAAGACTAGGAGGCCCAG", "GGCCTCCTAGTCTTCTGTGGTGGAATTGCG"]:
    change_block(seq,"CAG_CTRL")
    
for seq in ["CTGGCGAGACAAGGCGAAGACATAGCGCTGGGC", "CAGCGCTATGTCTTCGCCTTGTCTCGCCAG"]:
    change_block(seq,"GCC_CTRL")
    
for seq in ["CGCAATTCCACCACAGAAGACTAGGAGGCC", "CTCCTAGTCTTCTGTGGTGGAATTGCG"]:
    change_block(seq,"prefix")
    
for seq in ["CTGGCGAGACAAGGCGAAGACATAGCGCTG", "CGCTATGTCTTCGCCTTGTCTCGCCAG"]:
    change_block(seq,"suffix")

blocks = blocks_in_dataframe(df)

In [120]:

new_plate = []
for block in blocks:
    new_plate += [[block, find_wells('Block',block)]]
destination = pd.DataFrame(new_plate, columns = ['Block', 'From'])
destination = destination.reindex(destination['Block'].str.len().sort_values().index).reset_index(drop=True) # reorganize
destination['Well'] = destination.index



In [133]:
robot.connect("/dev/ttyACM0")

In [136]:
locations = {
        "tiprack-200" : "A3",
        "tiprack-10_2" : "E3",
        "tiprack-10_3" : "E2",
        "tiprack-10_1" : "E1",
        "trash" : "D1",
        "PCR-strip-tall" : "C3",
        "Transformation" : "C2",
        "Tube_rack" : "B1"
     }

def robo_locations(locations):

    # Make the dataframe to represent the OT-1 deck
    deck = ['A1','B2','C3','D2','E1']
    slots = pd.Series(deck)
    columns = sorted(slots.str[0].unique())
    rows = sorted(slots.str[1].unique(), reverse=True)
    layout_table = pd.DataFrame(index=rows, columns=columns)
    layout_table.fillna("---", inplace=True)

    # Fill in the data frame with the locations
    for obj in locations:
            layout_table.loc[locations[obj][1], locations[obj][0]] = obj

    # Displays the required plate map and waits to proceed
    print("\n Please arrange the items in the following configuration: \n")
    print(layout_table,"\n")
    input("Press enter to continue")
    

robo_locations(locations)


 Please arrange the items in the following configuration: 

             A          B               C      D             E
3  tiprack-200        ---  PCR-strip-tall    ---  tiprack-10_2
2          ---        ---  Transformation    ---  tiprack-10_3
1          ---  Tube_rack             ---  trash  tiprack-10_1 

Press enter to continue
